<a href="https://colab.research.google.com/github/luisosmx/Python_exercises/blob/main/Cloud_storage_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-storage
!pip install pandas google-cloud-storage

In [8]:
from google.cloud import storage
import os
import csv
import json


# Configura la variable de entorno para las credenciales
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/path/file.json'

# Crea un cliente de almacenamiento
client = storage.Client()

bucket_name = 'bucketdata_storage'
blob_name = 'Declaratorias2000_2016.csv'
destination_file_name = 'file.csv'

# Descargar el archivo CSV desde Cloud Storage
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.download_to_filename(destination_file_name)

# Convertir el archivo CSV a JSON
csv_data = []
with open(destination_file_name, 'r', encoding='ISO-8859-1') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        csv_data.append(row)



keys_json = {'Identificador': 'id',
             'Clave Estado': 'Clave_Estado',
             'Clave Municipio': 'Clave_Municipio',
             'Tipo Declaratoria': 'Tipo_Declaratoria',
             'Clasificación Fenómeno': 'Clasificacion_fenomeno',
             'Tipo Fenómeno': 'Tipo_Fenomeno',
             'Fecha Inicio': 'Fecha_Inicio',
             'Fecha Fin': 'Fecha_Fin'
             }

for item in csv_data:
    for old_key, new_key in keys_json.items():
        if old_key in item:
            item[new_key] = item.pop(old_key)

modified_json = json.dumps(csv_data, indent=4)

#print(modified_json)

In [9]:
from google.cloud import bigquery
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/path/file.json'

# Define el ID del proyecto y el ID del conjunto de datos
project_id = 'data-project-395314'
dataset_id = 'data_project'
table_id = 'declaratorias'

# Define el esquema de la tabla
schema = [
    bigquery.SchemaField("Estado", "STRING"),
    bigquery.SchemaField("Municipio", "STRING"),
    bigquery.SchemaField("Fecha Publicación", "STRING"),
    bigquery.SchemaField("Observaciones", "STRING"),
    bigquery.SchemaField("id", "STRING"),
    bigquery.SchemaField("Clave_Estado", "STRING"),
    bigquery.SchemaField("Clave_Municipio", "STRING"),
    bigquery.SchemaField("Tipo_Declaratoria", "STRING"),
    bigquery.SchemaField("Clasificacion_fenomeno", "STRING"),
    bigquery.SchemaField("Tipo_Fenomeno", "STRING"),
    bigquery.SchemaField("Fecha_Inicio", "STRING"),
    bigquery.SchemaField("Fecha_Fin", "STRING")
]

data = json.loads(modified_json)

# Configura tu cliente de BigQuery
client = bigquery.Client()

# Obtiene una referencia a la tabla existente
table_ref = client.dataset(dataset_id).table(table_id)

# Carga los datos en la tabla
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND  # Puedes ajustar esto según tus necesidades
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON

# Carga los datos en la tabla existente
load_job = client.load_table_from_json(data, table_ref, job_config=job_config)
load_job.result()  # Espera a que se complete la carga

print("Carga completada en la tabla: {}.{}".format(dataset_id, table_id))

Carga completada en la tabla: data_project.declaratorias
